In [1]:
import argparse
import os
import time
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim
import fastText

os.environ['CUDA_VISIBLE_DEVICES']="4,5,6,7"

In [2]:
from models import models
from dataset import openimages
from utils.loss import HardNegativeContrastiveLoss

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [5]:
def train(train_loader, model, criterion, optimizer, epoch, print_freq=1000):
    #amp_handle = amp.init()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    model = model.train()
    print("Start training")
    end = time.time()
    for i, (imgs, caps) in enumerate(train_loader):
        if i%2 == 1:
                print("%2.2f"% (i/len(train_loader)*100), '\%', end='\r')
        input_imgs, target = imgs.cuda(), caps.cuda()
        

        data_time.update(time.time() - end)

        optimizer.zero_grad()
        
        output_imgs = model(input_imgs)
        
        
        loss = criterion(output_imgs, target)
        
        #with amp_handle.scale_loss(loss, optimizer) as scaled_loss:
        #    scaled_loss.backward()
        loss.backward()
        optimizer.step()
        
        losses.update(loss.item(), imgs.size(0))

        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0 or i == (len(train_loader) - 1):
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses))

    return losses.avg, batch_time.avg, data_time.avg

In [6]:
def validate(val_loader, model, criterion, print_freq=1000):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    model = model.eval()

    imgs_enc = list()
    caps_enc = list()
    end = time.time()
    for i, (imgs, caps, lengths) in enumerate(val_loader):

        input_imgs, input_caps = imgs.cuda(), caps.cuda()

        # measure data loading time
        data_time.update(time.time() - end)

        with torch.no_grad():
            output_imgs = model(input_imgs)
            loss = criterion(output_imgs, input_caps)

        imgs_enc.append(output_imgs.cpu().data.numpy())
        caps_enc.append(output_caps.cpu().data.numpy())
        losses.update(loss.item(), imgs.size(0))

        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0 or i == (len(val_loader) - 1):
            print('Data: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                      i, len(val_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses))

    recall  = eval_recall(imgs_enc, caps_enc)
    print(recall)
    return losses.avg, batch_time.avg, data_time.avg, recall

In [7]:
normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

prepro = transforms.Compose([
    transforms.RandomResizedCrop(224),

    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

prepro_val = transforms.Compose([
    transforms.Resize((350, 350)),
    transforms.ToTensor(),
    normalize,
])

In [8]:
m = nn.DataParallel(models.ImageProjection().train().cuda())

In [9]:
for params in m.parameters():
    params.requires_grad=False

In [10]:
for params in m.module.projection.parameters():
    params.requires_grad=True

In [11]:
def collate_embeds(data):
    images, targets = zip(*data)
    images = torch.stack(images, 0)
    targets = torch.Tensor(np.stack(targets, 0))

    return images, targets

In [12]:
embed = fastText.load_model("/data/m.portaz/wiki.en.bin")
train_dataset = openimages.OpenImagesText(image_dir="/data/datasets/openimages/images/train/", 
                          dataset_file="/data/datasets/openimages/train-words.csv",
                          embeddings=embed, 
                          transform=prepro, random=0.5)

Reading dataset file
Done reading  4593616  lines.


In [13]:
train_loader = DataLoader(train_dataset, batch_size=3072, shuffle=True, drop_last=True,
                            num_workers=20, collate_fn=collate_embeds, pin_memory=True)

In [14]:
opti = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.001)

In [16]:
criterion = HardNegativeContrastiveLoss().cuda()

In [18]:
for i in range(2):
    train(train_loader, m, criterion, opti, i, print_freq=50)

Start training
Epoch: [0][0/1495]	Time 57.686 (57.686)	Data 55.812 (55.812)	Loss 11156.9834 (11156.9834)	
Epoch: [0][50/1495]	Time 1.941 (3.176)	Data 0.172 (1.323)	Loss 14512.1816 (11738.2635)	
Epoch: [0][100/1495]	Time 1.930 (2.611)	Data 0.180 (0.763)	Loss 10697.8145 (12910.2606)	
Epoch: [0][150/1495]	Time 1.927 (2.428)	Data 0.174 (0.572)	Loss 9919.4033 (12393.1876)	
Epoch: [0][200/1495]	Time 2.356 (2.334)	Data 0.171 (0.474)	Loss 9541.3018 (11695.1548)	
Epoch: [0][250/1495]	Time 1.931 (2.278)	Data 0.174 (0.415)	Loss 11456.1816 (11713.4279)	
Epoch: [0][300/1495]	Time 1.930 (2.242)	Data 0.173 (0.378)	Loss 8610.6426 (11527.3202)	
Epoch: [0][350/1495]	Time 1.927 (2.212)	Data 0.173 (0.349)	Loss 10150.8496 (11256.3370)	
Epoch: [0][400/1495]	Time 1.932 (2.191)	Data 0.173 (0.329)	Loss 8981.0293 (11009.7895)	
Epoch: [0][450/1495]	Time 1.928 (2.171)	Data 0.172 (0.312)	Loss 7935.8086 (10773.1899)	
Epoch: [0][500/1495]	Time 1.928 (2.158)	Data 0.174 (0.299)	Loss 11941.0254 (10590.1031)	
Epoch: [0]

(11155.130577510974, 2.0784580095157175, 0.2216013138110821)

In [21]:
opti = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=0.00025)

In [22]:
train(train_loader, m, criterion, opti, 4, print_freq=50)

Start training
Epoch: [4][0/1495]	Time 61.458 (61.458)	Data 59.626 (59.626)	Loss 8671.2266 (8671.2266)	
Epoch: [4][50/1495]	Time 1.913 (3.199)	Data 0.169 (1.360)	Loss 4107.7988 (5924.8596)	
Epoch: [4][100/1495]	Time 1.931 (2.637)	Data 0.170 (0.783)	Loss 3169.9741 (4811.2926)	
Epoch: [4][150/1495]	Time 2.574 (2.453)	Data 0.169 (0.584)	Loss 2787.2908 (4196.5531)	
Epoch: [4][200/1495]	Time 1.940 (2.359)	Data 0.186 (0.485)	Loss 2699.8037 (3825.2861)	
Epoch: [4][250/1495]	Time 1.916 (2.297)	Data 0.171 (0.424)	Loss 2807.6538 (3594.3144)	
Epoch: [4][300/1495]	Time 2.076 (2.253)	Data 0.300 (0.384)	Loss 2800.8647 (3443.1368)	
Epoch: [4][350/1495]	Time 1.920 (2.234)	Data 0.170 (0.356)	Loss 2857.4600 (3352.7536)	
Epoch: [4][400/1495]	Time 1.920 (2.210)	Data 0.170 (0.335)	Loss 3260.1409 (3334.5414)	
Epoch: [4][450/1495]	Time 2.116 (2.191)	Data 0.364 (0.319)	Loss 3015.0947 (3299.1341)	
Epoch: [4][500/1495]	Time 2.449 (2.176)	Data 0.169 (0.305)	Loss 2735.7253 (3249.4279)	
Epoch: [4][550/1495]	Time 1

Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/envs/py36/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/opt/conda/envs/py36/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/envs/py36/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/envs/py36/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/opt/conda/envs/py36/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/opt/conda/envs/py36/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/envs/py36/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + 

KeyboardInterrupt: 

In [21]:
for param in m.parameters():
    param.requires_grad = True

# Keep the first layer of resnet frozen
for i in range(0, 6):
    for param in m.module.base_layer[i].parameters():
        param.requires_grad = False

In [28]:
train_loader = DataLoader(train_dataset, batch_size=450, shuffle=True, drop_last=True,
                            num_workers=20, collate_fn=collate_embeds, pin_memory=True)
opti = optim.Adam(filter(lambda p: p.requires_grad, m.module.parameters()), lr=0.00005)

In [29]:
for _ in range(5):
    train(train_loader, m, criterion, opti, i, print_freq=100)
    i += 1

Start training
Epoch: [8][0/10208]	Time 15.876 (15.876)	Data 14.522 (14.522)	Loss 249.2530 (249.2530)	
Epoch: [8][100/10208]	Time 0.724 (0.927)	Data 0.026 (0.171)	Loss 226.3943 (239.8907)	
Epoch: [8][200/10208]	Time 0.724 (0.868)	Data 0.026 (0.099)	Loss 228.6726 (235.6185)	
Epoch: [8][300/10208]	Time 0.725 (0.838)	Data 0.026 (0.075)	Loss 230.6595 (233.0092)	
Epoch: [8][400/10208]	Time 0.725 (0.822)	Data 0.026 (0.063)	Loss 225.4995 (231.3337)	
Epoch: [8][500/10208]	Time 0.945 (0.813)	Data 0.026 (0.056)	Loss 217.3611 (229.5482)	
Epoch: [8][600/10208]	Time 0.730 (0.809)	Data 0.026 (0.051)	Loss 215.5688 (228.3176)	
Epoch: [8][700/10208]	Time 0.731 (0.803)	Data 0.026 (0.047)	Loss 227.7986 (227.4418)	
Epoch: [8][800/10208]	Time 0.763 (0.801)	Data 0.025 (0.045)	Loss 213.1459 (226.4875)	
Epoch: [8][900/10208]	Time 0.735 (0.798)	Data 0.026 (0.043)	Loss 219.0038 (225.6315)	
Epoch: [8][1000/10208]	Time 0.828 (0.797)	Data 0.027 (0.041)	Loss 214.3507 (224.8640)	
Epoch: [8][1100/10208]	Time 0.764 (0

Epoch: [9][8400/10208]	Time 0.720 (0.778)	Data 0.025 (0.028)	Loss 186.1743 (189.0974)	
Epoch: [9][8500/10208]	Time 0.737 (0.778)	Data 0.025 (0.028)	Loss 185.1117 (189.0482)	
Epoch: [9][8600/10208]	Time 0.739 (0.778)	Data 0.025 (0.028)	Loss 183.8793 (189.0015)	
Epoch: [9][8700/10208]	Time 0.757 (0.778)	Data 0.026 (0.028)	Loss 186.3570 (188.9567)	
Epoch: [9][8800/10208]	Time 1.146 (0.778)	Data 0.026 (0.028)	Loss 184.0963 (188.9085)	
Epoch: [9][8900/10208]	Time 0.730 (0.778)	Data 0.025 (0.028)	Loss 187.3147 (188.8627)	
Epoch: [9][9000/10208]	Time 0.758 (0.778)	Data 0.026 (0.028)	Loss 184.9234 (188.8192)	
Epoch: [9][9100/10208]	Time 0.776 (0.778)	Data 0.050 (0.028)	Loss 185.4924 (188.7735)	
Epoch: [9][9200/10208]	Time 0.747 (0.778)	Data 0.025 (0.028)	Loss 184.0836 (188.7268)	
Epoch: [9][9300/10208]	Time 1.078 (0.778)	Data 0.025 (0.028)	Loss 183.1010 (188.6787)	
Epoch: [9][9400/10208]	Time 0.727 (0.778)	Data 0.026 (0.028)	Loss 182.8998 (188.6328)	
Epoch: [9][9500/10208]	Time 0.722 (0.778)	D

Epoch: [10][7400/10208]	Time 0.730 (0.779)	Data 0.026 (0.028)	Loss 179.0836 (181.2811)	
Epoch: [10][7500/10208]	Time 0.724 (0.779)	Data 0.026 (0.028)	Loss 181.0837 (181.2523)	
Epoch: [10][7600/10208]	Time 0.728 (0.779)	Data 0.026 (0.028)	Loss 177.7071 (181.2237)	
Epoch: [10][7700/10208]	Time 1.115 (0.779)	Data 0.026 (0.028)	Loss 177.9585 (181.1977)	
Epoch: [10][7800/10208]	Time 0.728 (0.779)	Data 0.026 (0.028)	Loss 177.7560 (181.1708)	
Epoch: [10][7900/10208]	Time 0.722 (0.779)	Data 0.027 (0.028)	Loss 176.2551 (181.1457)	
Epoch: [10][8000/10208]	Time 0.730 (0.779)	Data 0.025 (0.028)	Loss 179.4148 (181.1177)	
Epoch: [10][8100/10208]	Time 0.725 (0.779)	Data 0.026 (0.028)	Loss 181.2314 (181.0873)	
Epoch: [10][8200/10208]	Time 1.095 (0.779)	Data 0.030 (0.028)	Loss 178.5480 (181.0590)	
Epoch: [10][8300/10208]	Time 0.725 (0.779)	Data 0.026 (0.028)	Loss 178.9069 (181.0326)	
Epoch: [10][8400/10208]	Time 0.740 (0.779)	Data 0.026 (0.028)	Loss 181.1436 (181.0055)	
Epoch: [10][8500/10208]	Time 0.7

Epoch: [11][6300/10208]	Time 0.744 (0.780)	Data 0.025 (0.029)	Loss 177.9902 (177.9877)	
Epoch: [11][6400/10208]	Time 0.758 (0.780)	Data 0.025 (0.029)	Loss 178.8732 (177.9820)	
Epoch: [11][6500/10208]	Time 0.725 (0.780)	Data 0.026 (0.029)	Loss 175.0977 (177.9756)	
Epoch: [11][6600/10208]	Time 1.238 (0.780)	Data 0.025 (0.029)	Loss 179.1657 (177.9703)	
Epoch: [11][6700/10208]	Time 0.724 (0.780)	Data 0.025 (0.029)	Loss 178.7919 (177.9649)	
Epoch: [11][6800/10208]	Time 0.721 (0.779)	Data 0.026 (0.029)	Loss 177.0470 (177.9617)	
Epoch: [11][6900/10208]	Time 0.728 (0.780)	Data 0.026 (0.029)	Loss 177.3727 (177.9559)	
Epoch: [11][7000/10208]	Time 0.723 (0.779)	Data 0.025 (0.029)	Loss 181.3313 (177.9536)	
Epoch: [11][7100/10208]	Time 1.149 (0.779)	Data 0.026 (0.029)	Loss 179.8049 (177.9482)	
Epoch: [11][7200/10208]	Time 0.724 (0.779)	Data 0.025 (0.029)	Loss 179.0040 (177.9407)	
Epoch: [11][7300/10208]	Time 0.721 (0.779)	Data 0.025 (0.029)	Loss 176.4835 (177.9369)	
Epoch: [11][7400/10208]	Time 0.7

Epoch: [12][5200/10208]	Time 0.719 (0.780)	Data 0.026 (0.029)	Loss 175.7195 (177.1066)	
Epoch: [12][5300/10208]	Time 0.730 (0.780)	Data 0.026 (0.029)	Loss 177.7201 (177.1062)	
Epoch: [12][5400/10208]	Time 0.725 (0.780)	Data 0.025 (0.029)	Loss 178.3834 (177.1069)	
Epoch: [12][5500/10208]	Time 1.104 (0.780)	Data 0.026 (0.029)	Loss 177.8806 (177.1075)	
Epoch: [12][5600/10208]	Time 0.744 (0.780)	Data 0.025 (0.029)	Loss 175.8921 (177.1048)	
Epoch: [12][5700/10208]	Time 0.717 (0.780)	Data 0.026 (0.029)	Loss 174.6758 (177.0996)	
Epoch: [12][5800/10208]	Time 0.728 (0.780)	Data 0.026 (0.029)	Loss 176.9340 (177.0971)	
Epoch: [12][5900/10208]	Time 0.761 (0.780)	Data 0.026 (0.029)	Loss 176.7609 (177.0936)	
Epoch: [12][6000/10208]	Time 1.108 (0.780)	Data 0.026 (0.029)	Loss 176.2689 (177.0906)	
Epoch: [12][6100/10208]	Time 0.738 (0.780)	Data 0.026 (0.029)	Loss 177.6653 (177.0885)	
Epoch: [12][6200/10208]	Time 0.724 (0.780)	Data 0.026 (0.029)	Loss 178.6955 (177.0817)	
Epoch: [12][6300/10208]	Time 0.7

In [35]:
embed = fastText.load_model("/data/m.portaz/wiki.en.bin")
train_dataset = openimages.OpenImagesText(image_dir="/data/datasets/openimages/images/train/", 
                          dataset_file="/data/datasets/openimages/train-words.csv",
                          embeddings=embed, 
                          transform=prepro, random=0.5)

Reading dataset file
Done reading  4593616  lines.


In [42]:
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, drop_last=True,
                            num_workers=20, collate_fn=collate_embeds, pin_memory=True)
opti = optim.Adam(filter(lambda p: p.requires_grad, m.module.parameters()), lr=0.000025)

In [43]:
for _ in range(1):
    train(train_loader, m, criterion, opti, i, print_freq=200)
    i += 1

Start training


RuntimeError: CUDA out of memory. Tried to allocate 73.50 MiB (GPU 3; 15.75 GiB total capacity; 14.25 GiB already allocated; 61.94 MiB free; 265.65 MiB cached) (malloc at /opt/conda/conda-bld/pytorch_1549630534704/work/aten/src/THC/THCCachingAllocator.cpp:231)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x45 (0x7fe1d82abcf5 in /opt/conda/envs/py36/lib/python3.6/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x1239bc1 (0x7fe184f71bc1 in /opt/conda/envs/py36/lib/python3.6/site-packages/torch/lib/libcaffe2_gpu.so)
frame #2: <unknown function> + 0x123a53a (0x7fe184f7253a in /opt/conda/envs/py36/lib/python3.6/site-packages/torch/lib/libcaffe2_gpu.so)
frame #3: at::native::empty_cuda(c10::ArrayRef<long>, at::TensorOptions const&) + 0x2d6 (0x7fe1865dcdb6 in /opt/conda/envs/py36/lib/python3.6/site-packages/torch/lib/libcaffe2_gpu.so)
frame #4: at::CUDAFloatType::empty(c10::ArrayRef<long>, at::TensorOptions const&) const + 0x161 (0x7fe184e90311 in /